In [1]:
import os
os.chdir("/tf/src/examples/greetings")

from greetings import Dataset
import tensorflow as tf
import numpy as np
import logging
from greetings import GreetingModel, cfg
from copynet_tf.loss import CopyNetLoss
from copynet_tf.metrics import BLEU
logging.basicConfig(
        level=cfg.LOG_LVL,
        filename=cfg.LOG_FILENAME,
        format='%(message)s')

In [2]:
data = Dataset()

In [3]:
model = GreetingModel()

In [4]:
RNG_SEED = 11
to_gpu = tf.data.experimental.copy_to_device("/gpu:0")
data = data.train.shuffle(
    buffer_size=10000, seed=RNG_SEED, reshuffle_each_iteration=False)
train = data.skip(512).batch(512, drop_remainder=True).apply(to_gpu)
val = data.take(512).batch(512, drop_remainder=True).apply(to_gpu)
with tf.device("/gpu:0"):
    train = train.prefetch(3)
    val = val.prefetch(3)

In [5]:
X, y = next(val.as_numpy_iterator())
print("source idx from source vocab\n", X[0][:3])
print("\nsource idx from target vocab\n", X[1][:3])
print("\ntarget idx from target vocab\n", y[0][:3])
print("\ntarget idx from source vocab\n", y[1][:3])

source idx from source vocab
 [[   2   11   29    4   15    7 2723  773    4    3]
 [   2   22   75    8   14    4    3    0    0    0]
 [   2   18   28    6    5  232 4784    3    0    0]]

source idx from target vocab
 [[ 2  1  1  1  1  1  1  1  1  3]
 [ 2  1 54  8  1  1  3  0  0  0]
 [ 2  1  1  1  1  1  1  3  0  0]]

target idx from target vocab
 [[ 2  4  5  6  7  8  1  1  9  3]
 [ 2  4  5  6  7  8 54  9  3  0]
 [ 2  4  5  6  7  8  1  1  9  3]]

target idx from source vocab
 [[   2    1    1    1    1    8 2723  773    1    3]
 [   2    1    1    1    1    8   75    1    3    0]
 [   2    1    1    1    1    8  232 4784    1    3]]


In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3, clipnorm=cfg.CLIP_NORM),
    loss=CopyNetLoss(),
    metrics=[
        BLEU(ignore_tokens=[0,2,3], ignore_all_tokens_after=3),
        BLEU(ignore_tokens=[0,2,3], ignore_all_tokens_after=3, name='bleu-smooth', smooth=True)])

In [7]:
hist = model.fit(train, epochs=5, validation_data=val, shuffle=False)

Epoch 1/5
8/8 [==============================] - 5s 683ms/step - loss: 1.3554 - bleu: 0.5098 - bleu-smooth: 0.5098 - val_loss: 0.1878 - val_bleu: 0.8235 - val_bleu-smooth: 0.8235
Epoch 2/5
8/8 [==============================] - 4s 458ms/step - loss: 0.1361 - bleu: 0.8692 - bleu-smooth: 0.8692 - val_loss: 0.0701 - val_bleu: 0.9381 - val_bleu-smooth: 0.9381
Epoch 3/5
8/8 [==============================] - 4s 469ms/step - loss: 0.0407 - bleu: 0.9680 - bleu-smooth: 0.9680 - val_loss: 0.0210 - val_bleu: 0.9858 - val_bleu-smooth: 0.9858
Epoch 4/5
8/8 [==============================] - 4s 473ms/step - loss: 0.0113 - bleu: 0.9937 - bleu-smooth: 0.9937 - val_loss: 0.0075 - val_bleu: 0.9951 - val_bleu-smooth: 0.9951
Epoch 5/5
8/8 [==============================] - 4s 481ms/step - loss: 0.0045 - bleu: 0.9979 - bleu-smooth: 0.9979 - val_loss: 0.0048 - val_bleu: 0.9974 - val_bleu-smooth: 0.9974


In [8]:
def idx2str(pred_y, X):
    ret = []
    vocab_len = model.vocab.get_vocab_size("target")
    for idx in pred_y:
        if idx < vocab_len:
            ret.append(model.vocab.get_token_text(idx, "target"))
        else:
            ret.append(model.vocab.get_token_text(X[idx-vocab_len], "source"))
    return ret

In [9]:
pred = model.predict(train)
pred, pred_proba = pred["predictions"], pred["predicted_probas"]
for i, Xy in enumerate(train.unbatch().take(10)):
    X, y = Xy
    s = model.vocab.inverse_transform(X[0].numpy()[np.newaxis, :], "source")[0]
    t = model.vocab.inverse_transform(y[0].numpy()[np.newaxis, :], "target")[0]
    print(f"Source: {' '.join(s)}\nTarget: {' '.join(t)}\n")
    for j in range(10):
        p = idx2str(pred[i][j], X[0].numpy())
        print(f"Predicted: {' '.join(p)}\tProba: {tf.exp(pred_proba[i][j])}")
    print("")

Source: <S> Call me Alma Poole EOS PAD PAD PAD PAD
Target: <S> Nice to meet you , UNKNOWN UNKNOWN ! EOS

Predicted: Nice to meet you , Alma Poole ! EOS	Proba: 0.9968473315238953
Predicted: Nice to meet you , Alma ! EOS EOS	Proba: 0.0014781506033614278
Predicted: Nice to meet you Alma Poole ! EOS EOS	Proba: 0.000958629825618118
Predicted: to meet you , Alma Poole ! EOS EOS	Proba: 0.00020651805971283466
Predicted: Nice meet you , Alma Poole ! EOS EOS	Proba: 0.00013739355199504644
Predicted: Nice to you , Alma Poole ! EOS EOS	Proba: 0.00012071990931872278
Predicted: Nice to meet you , Poole ! EOS EOS	Proba: 5.467259688884951e-05
Predicted: Nice to meet , Alma Poole ! EOS EOS	Proba: 5.102485010866076e-05
Predicted: Nice to meet you , Alma Alma ! EOS	Proba: 3.107051088591106e-05
Predicted: Nice to meet you , Alma EOS ! EOS	Proba: 2.0938558009220287e-05

Source: <S> My friends call me Nathanael EOS PAD PAD PAD
Target: <S> Nice to meet you , UNKNOWN ! EOS PAD

Predicted: Nice to meet you , Na

In [10]:
pred = model.predict(val.unbatch().take(50).batch(5, drop_remainder=True))
pred, pred_proba = pred["predictions"], pred["predicted_probas"]
for i, Xy in enumerate(val.unbatch().take(10)):
    X, y = Xy
    s = model.vocab.inverse_transform(X[0].numpy()[np.newaxis, :], "source")[0]
    t = model.vocab.inverse_transform(y[0].numpy()[np.newaxis, :], "target")[0]
    print(f"Source: {' '.join(s)}\nTarget: {' '.join(t)}\n")
    for j in range(10):
        p = idx2str(pred[i][j], X[0].numpy())
        print(f"Predicted: {' '.join(p)}\tProba: {tf.exp(pred_proba[i][j])}")
    print("")

Source: <S> Hi there . It 's Nyla Wais . EOS
Target: <S> Nice to meet you , UNKNOWN UNKNOWN ! EOS

Predicted: Nice to meet you , Nyla Wais ! EOS	Proba: 0.9964045286178589
Predicted: Nice to meet you Nyla Wais ! EOS EOS	Proba: 0.002326371381059289
Predicted: Nice to meet you , Nyla . ! EOS	Proba: 0.0002857318613678217
Predicted: Nice meet you , Nyla Wais ! EOS EOS	Proba: 0.00026062471442855895
Predicted: Nice to meet , Nyla Wais ! EOS EOS	Proba: 0.00017058607772924006
Predicted: Nice to you , Nyla Wais ! EOS EOS	Proba: 0.00014674717385787517
Predicted: to meet you , Nyla Wais ! EOS EOS	Proba: 0.00013336109986994416
Predicted: Nice to meet you , Nyla Wais Wais !	Proba: 0.00010054439189843833
Predicted: Nice to meet you , Nyla Wais . !	Proba: 5.6527103879489005e-05
Predicted: Nice to meet you , Nyla ! EOS EOS	Proba: 2.9029641154920682e-05

Source: <S> Its Roseann , actually . EOS PAD PAD PAD
Target: <S> Nice to meet you , Roseann ! EOS PAD

Predicted: Nice to meet you , Roseann ! EOS EOS	